# **Preparación de datos para `KPIs`**

### Para la realización de los mismos, en primer lugar procedemos a obtener la cantidad de habitantes en CABA en el período comprendido entre los años 2016 y 2021. Para ello nos basamos de los datos obtenidos en el sitio web de la Dirección General de Estadística y Censos de la Ciudad de Buenos Aires, los cuales pueden ser descargados en el siguiente [link](https://data.buenosaires.gob.ar/dataset/estructura-poblacion)

### Importamos las librerías necesarias para el análisis de los datos.

In [1]:
import pandas as pd

In [2]:
pob_2016 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2016')
pob_2017 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2017')
pob_2018 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2018')
pob_2019 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2019')
pob_2020 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2020')
pob_2021 = pd.read_excel('Poblacion_CABA/PBP_CO1025.xls',sheet_name = '2021')

### Visualizamos los datos obtenidos para corroborar que los mismos se encuentren en condiciones de ser utilizados y obtener la ubicación del que nos interesa.

In [3]:
pob_2016.head(3)

,Proyecciones de población por grupo de edad y comuna según sexo. Ciudad de Buenos Aires. 1 de julio de 2016,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grupo de edad (años),Total,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15
2,Total,3059122,252053.0,149848.0,192573.0,238303.0,186740.0,184611.0,240116.0,225737.0,170353.0,170163.0,189956.0,213576.0,235819.0,226884.0,182390


### Extraemos la cantidad de habitantes de CABA por año y lo encerramos en una variable.

In [4]:
hab_2016 = pob_2016['Unnamed: 1'][2]
hab_2017 = pob_2017['Unnamed: 1'][2]
hab_2018 = pob_2018['Unnamed: 1'][2]
hab_2019 = pob_2019['Unnamed: 1'][2]
hab_2020 = pob_2020['Unnamed: 1'][2]
hab_2021 = pob_2021['Unnamed: 1'][2]

In [5]:
# Revisamos que los datos sean correctos
hab_2016

3059122

### Preparamos los datasets para la realización de los `KPIs` y los guardamos en archivos `.xlsx` para su posterior utilización.

#### + Primer Dataset

In [6]:
accidentes_CABA = pd.read_excel('accidentes.xlsx')
accidentes_CABA.head(3)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_VICTIMA,VEHICULO_ACUSADO,GRAVEDAD,CARATULA_POLICIAL
0,2016-0001,1,2016-01-01,4.0,AVENIDA,8.0,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO,AUTO,FATAL,HOMICIDIO CULPOSO
1,2016-0002,1,2016-01-02,1.0,AVENIDA,9.0,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO,PASAJEROS,FATAL,HOMICIDIO CULPOSO
2,2016-0003,1,2016-01-03,7.0,AVENIDA,1.0,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO,AUTO,FATAL,HOMICIDIO CULPOSO


In [7]:
# Cambiamos el tipo de dato de FECHA y mantenemos solo las filas donde GRAVEDAD sea FATAL
accidentes_CABA['FECHA'] = pd.to_datetime(accidentes_CABA['FECHA'])
accidentes_CABA = accidentes_CABA[accidentes_CABA['GRAVEDAD'] == 'FATAL']

In [8]:
# Creamos columnas de AÑO y SEMESTRE
accidentes_CABA['AÑO'] = accidentes_CABA['FECHA'].dt.year
accidentes_CABA['FATAL SEM 1'] = ((accidentes_CABA['FECHA'].dt.month <= 6).astype(int) * 1)
accidentes_CABA['FATAL SEM 2'] = ((accidentes_CABA['FECHA'].dt.month > 6).astype(int) * 1)

In [9]:
accidentes_CABA = (accidentes_CABA.groupby('AÑO').agg({'FATAL SEM 1': 'sum', 'FATAL SEM 2': 'sum'}).reset_index())

In [10]:
accidentes_CABA

,AÑO,FATAL SEM 1,FATAL SEM 2
0,2016,64,80
1,2017,64,67
2,2018,67,76
3,2019,56,47
4,2020,32,49
5,2021,55,42


In [11]:
# Creamos la nueva columna 'TOTAL HABITANTES' en el DataFrame accidentes_CABA
accidentes_CABA['TOTAL HABITANTES'] = 0  # Inicializar la columna con ceros

# Asignamos el valor correspondiente a cada fila en función del año
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2016, 'TOTAL HABITANTES'] = hab_2016
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2017, 'TOTAL HABITANTES'] = hab_2017
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2018, 'TOTAL HABITANTES'] = hab_2018
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2019, 'TOTAL HABITANTES'] = hab_2019
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2020, 'TOTAL HABITANTES'] = hab_2020
accidentes_CABA.loc[accidentes_CABA['AÑO'] == 2021, 'TOTAL HABITANTES'] = hab_2021

In [12]:
accidentes_CABA

,AÑO,FATAL SEM 1,FATAL SEM 2,TOTAL HABITANTES
0,2016,64,80,3059122
1,2017,64,67,3063728
2,2018,67,76,3068043
3,2019,56,47,3072029
4,2020,32,49,3075646
5,2021,55,42,3078836


In [57]:
# Guardamos el DataFrame en un archivo Excel
accidentes_CABA.to_excel('primer_kpi.xlsx', index=False)

#### + Segundo Dataset

In [13]:
fatal_moto = pd.read_excel('accidentes.xlsx')
fatal_moto.head(3)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_VICTIMA,VEHICULO_ACUSADO,GRAVEDAD,CARATULA_POLICIAL
0,2016-0001,1,2016-01-01,4.0,AVENIDA,8.0,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO,AUTO,FATAL,HOMICIDIO CULPOSO
1,2016-0002,1,2016-01-02,1.0,AVENIDA,9.0,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO,PASAJEROS,FATAL,HOMICIDIO CULPOSO
2,2016-0003,1,2016-01-03,7.0,AVENIDA,1.0,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO,AUTO,FATAL,HOMICIDIO CULPOSO


In [14]:
# Cambiamos el tipo de dato de FECHA y mantenemos solo las filas donde GRAVEDAD sea FATAL
fatal_moto['FECHA'] = pd.to_datetime(fatal_moto['FECHA'])
fatal_moto = fatal_moto[fatal_moto['GRAVEDAD'] == 'FATAL']

In [15]:
# Filtramos por VEHICULO_VICTIMA == MOTO
fatal_moto = fatal_moto[fatal_moto['VEHICULO_VICTIMA'] == 'MOTO']

In [16]:
# Creamos columna de AÑO
fatal_moto['AÑO'] = fatal_moto['FECHA'].dt.year

In [17]:
#Agrupamos por año
fatal_moto = fatal_moto.groupby('AÑO').size().reset_index(name='ACCIDENTES FATALES')

In [18]:
# Creamos la nueva columna 'TOTAL HABITANTES' en el DataFrame accidentes_CABA
fatal_moto['TOTAL HABITANTES'] = 0  # Inicializar la columna con ceros

# Asignamos el valor correspondiente a cada fila en función del año
fatal_moto.loc[fatal_moto['AÑO'] == 2016, 'TOTAL HABITANTES'] = hab_2016
fatal_moto.loc[fatal_moto['AÑO'] == 2017, 'TOTAL HABITANTES'] = hab_2017
fatal_moto.loc[fatal_moto['AÑO'] == 2018, 'TOTAL HABITANTES'] = hab_2018
fatal_moto.loc[fatal_moto['AÑO'] == 2019, 'TOTAL HABITANTES'] = hab_2019
fatal_moto.loc[fatal_moto['AÑO'] == 2020, 'TOTAL HABITANTES'] = hab_2020
fatal_moto.loc[fatal_moto['AÑO'] == 2021, 'TOTAL HABITANTES'] = hab_2021

In [19]:
fatal_moto

,AÑO,ACCIDENTES FATALES,TOTAL HABITANTES
0,2016,64,3059122
1,2017,53,3063728
2,2018,55,3068043
3,2019,50,3072029
4,2020,29,3075646
5,2021,46,3078836


### _`1er KPI`_: Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

In [58]:
# Calculamos la tasa de homicidios para cada semestre
accidentes_CABA['TASA SEM 1'] = (accidentes_CABA['FATAL SEM 1'] / accidentes_CABA['TOTAL HABITANTES']) * 100000
accidentes_CABA['TASA SEM 2'] = (accidentes_CABA['FATAL SEM 2'] / accidentes_CABA['TOTAL HABITANTES']) * 100000

# Calculamos la variación porcentual entre los semestres del mismo año
accidentes_CABA['REDUCCIÓN TASA'] = (
    (accidentes_CABA['TASA SEM 2'] - accidentes_CABA['TASA SEM 1']) / abs(accidentes_CABA['TASA SEM 1'])
) * 100

# Mostramos los resultados para el primer KPI
print(accidentes_CABA[['AÑO', 'FATAL SEM 1', 'FATAL SEM 2', 'TASA SEM 1', 'TASA SEM 2', 'REDUCCIÓN TASA']])

    AÑO  FATAL SEM 1  FATAL SEM 2  TASA SEM 1  TASA SEM 2  REDUCCIÓN TASA
0  2016           64           80    2.092104    2.615129       25.000000
1  2017           64           67    2.088958    2.186878        4.687500
2  2018           67           76    2.183803    2.477149       13.432836
3  2019           56           47    1.822899    1.529933      -16.071429
4  2020           32           49    1.040432    1.593161       53.125000
5  2021           55           42    1.786389    1.364152      -23.636364


### _`2do KPI`_: Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

In [30]:
# Calculamos la cantidad de accidentes mortales de motociclistas para el DataFrame fatal_moto
fatal_moto['ACCIDENTES FATAL MOTO'] = fatal_moto['ACCIDENTES FATALES']

# Calculamos la tasa de accidentes mortales de motociclistas
fatal_moto['TASA ACCIDENTES'] = (fatal_moto['ACCIDENTES FATAL MOTO'] / fatal_moto['TOTAL HABITANTES']) * 100000

# Calculamos la variación porcentual entre los años
fatal_moto['REDUCCION ACCIDENTES MOTO'] = ((fatal_moto['ACCIDENTES FATAL MOTO'].shift(-1) - fatal_moto['ACCIDENTES FATAL MOTO']) / fatal_moto['ACCIDENTES FATAL MOTO'].shift(-1)) * 100

# Mostramos los resultados para el segundo KPI
print(fatal_moto[['AÑO', 'ACCIDENTES FATAL MOTO', 'TASA ACCIDENTES', 'REDUCCION ACCIDENTES MOTO']])

    AÑO  ACCIDENTES FATAL MOTO  TASA ACCIDENTES  REDUCCION ACCIDENTES MOTO
0  2016                     64         2.092104                 -20.754717
1  2017                     53         1.729919                   3.636364
2  2018                     55         1.792674                 -10.000000
3  2019                     50         1.627589                 -72.413793
4  2020                     29         0.942891                  36.956522
5  2021                     46         1.494071                        NaN


In [22]:
fatal_moto.to_excel('segundo_kpi.xlsx', index=False)